# Исследование надежности заемщиков.

# Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

### Шаг 1. Обзор данных

In [34]:
import pandas as pd

In [35]:
data = pd.read_csv('data.csv')

In [36]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [38]:
data.value_counts()

children  days_employed   dob_years  education  education_id  family_status     family_status_id  gender  income_type  debt  total_income   purpose                              
-1        -9851.184337    44         среднее    1             в разводе         3                 F       сотрудник    0     110990.810581  свой автомобиль                          1
 1        -7117.338200    42         среднее    1             женат / замужем   0                 F       госслужащий  0     228199.252819  ремонт жилью                             1
          -7137.319761    35         среднее    1             женат / замужем   0                 F       сотрудник    0     129352.711834  недвижимость                             1
          -7146.299384    50         высшее     0             женат / замужем   0                 F       сотрудник    0     104888.625546  операции с жильем                        1
          -7152.420165    49         среднее    1             женат / замужем   0                 F       сотрудник    0     109556.390632  строительство жилой недвижимости         1
                                                                                                                                                                                    ..
 0        -1019.065321    45         среднее    1             женат / замужем   0                 F       компаньон    0     217241.515349  получение дополнительного образования    1
          -1019.319566    38         высшее     0             женат / замужем   0                 M       компаньон    0     141462.454320  покупка коммерческой недвижимости        1
          -1019.684185    62         среднее    1             вдовец / вдова    2                 F       сотрудник    0     81688.829006   операции с жильем                        1
          -1020.182597    42         среднее    1             женат / замужем   0                 M       сотрудник    0     142895.155108  строительство жилой недвижимости         1
 20        385267.263676  62         среднее    1             гражданский брак  1                 F       пенсионер    0     192221.076774  операции с коммерческой недвижимостью    1
Name: count, Length: 19351, dtype: int64

### Шаг 2.1 Заполнение пропусков

In [39]:
print(len(data[data['days_employed'].isna()]))
print(len(data[data['total_income'].isna()]))

2174
2174


In [40]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].mean())
data['total_income'] = data['total_income'].fillna(data['total_income'].mean())

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [41]:
data['days_employed'] = data['days_employed'].abs()
data['days_employed'] = data['days_employed'].apply(lambda x: x if x < 300000 else data['days_employed'].median())

data['children'] = data['children'].replace(-1, data['children'].median())
data['children'] = data['children'].replace(20, data['children'].median())

data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].median()

data['education'] = data['education'].str.lower()

### Шаг 2.3. Изменение типов данных.

In [42]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

### Шаг 2.4. Удаление дубликатов.

In [43]:
data = data.drop_duplicates()

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [44]:
education_dict = data[['education_id','education']]
family_dict = data[['family_status_id','family_status']]

education_dict = education_dict.drop_duplicates().reset_index(drop=True)
family_dict = family_dict.drop_duplicates().reset_index(drop=True)




data = data.reset_index(drop=True)


print(education_dict)
print(family_dict)

   education_id            education
0             0               высшее
1             1              среднее
2             2  неоконченное высшее
3             3            начальное
4             4       ученая степень
   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  Не женат / не замужем


### Шаг 2.6. Категоризация дохода.

In [58]:
def categorize_income(income):



    if income <= 100000:
        return 'низкий'
    if income > 100001 and income <= 130000:
        return 'средний'
    if income > 130001 and income <= 160000:
        return 'выше среднего'
    if income > 160001 and income <= 200000:
        return 'высокий'
    if income > 200001:
        return 'очень высокий'

data['total_income_category'] = data['total_income'].apply(categorize_income)


Есть ли зависимость между семейным положением и возвратом кредита в срок?
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Как разные цели кредита влияют на его возврат в срок?

### Шаг 2.7. Категоризация целей кредита.

In [46]:
def categorize_purpose(purpose):
    if 'автом' in purpose:
        return 'операции с автомобилем'
    elif 'жил' in purpose or 'недвиж' in purpose:
        return 'операции с недвижимостью'
    elif 'свад' in purpose:
        return 'проведение свадьбы'
    elif 'образов' in purpose:
        return 'получение образования'
    else:
        return 'другое'

data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [47]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,очень высокий,операции с недвижимостью
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,выше среднего,операции с автомобилем
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,выше среднего,операции с недвижимостью
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,очень высокий,получение образования
4,0,2609,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,высокий,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,очень высокий,операции с недвижимостью
21449,0,2609,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,выше среднего,операции с автомобилем
21450,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,средний,операции с недвижимостью
21451,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,очень высокий,операции с автомобилем


### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [48]:
data['children'] = data['children'].abs()

children_debt = data.pivot_table(index = ['children'], values='debt', aggfunc = ('mean', lambda X:
                                                                       X.count()))

def percent(x):
    return "{0:.2%}".format(x)

children_debt['mean'] = children_debt['mean'].apply(percent)
children_debt = children_debt.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее"})
children_debt.sort_values(by='Количество записей', ascending=False)

,Количество записей,Среднее
children,,
0,14213,7.54%
1,4808,9.23%
2,2052,9.45%
3,330,8.18%
4,41,9.76%
5,9,0.00%


Можно заметить, что чем меньше детей, то тем больше вероятность, что вернут кредит в срок


### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [49]:
family_debt = data.pivot_table(index = ['family_status'], values='debt', aggfunc = ('mean', lambda X:
                                                                       X.count()))

def percent(x):
    return "{0:.2%}".format(x)

family_debt['mean'] = family_debt['mean'].apply(percent)
family_debt = family_debt.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее"})
family_debt.sort_values(by='Количество записей', ascending=False)

,Количество записей,Среднее
family_status,,
женат / замужем,12339,7.55%
гражданский брак,4150,9.35%
Не женат / не замужем,2810,9.75%
в разводе,1195,7.11%
вдовец / вдова,959,6.57%


Можно заметить, что у вдова/вдовец возвращают кредит в срок, можно заметить, что далеко не ушли пара в разводе, так что можно было бы объеденить в один столбец.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [59]:
total_income_debt = data.pivot_table(index = ['total_income_category'], values='debt', aggfunc = ('mean', lambda X: X.count()))

def percent(x):
  return "{0:.2%}".format(x)

total_income_debt['mean'] = total_income_debt['mean'].apply(percent)
total_income_debt = total_income_debt.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее"})
total_income_debt.sort_values(by='Количество записей', ascending=False)

,Количество записей,Среднее
total_income_category,,
высокий,5254,8.56%
очень высокий,5065,7.07%
низкий,4463,7.93%
средний,3480,8.42%
выше среднего,3190,8.97%


Можно заметить, что люди у которых очень высокий доход(от 200000) закрывают кредит, также на 2 месте есть люди с низким, скорее всего это люди которые хотят улучшить свою кредитную историю.

### Как разные цели кредита влияют на его возврат в срок?

In [61]:
purpose_debt = data.pivot_table(index = ['purpose_category'], values='debt', aggfunc = ('mean', lambda X:
                                                                       X.count()))
def percent(x):
    return "{0:.2%}".format(x)

purpose_debt['mean'] = purpose_debt['mean'].apply(percent)
purpose_debt = purpose_debt.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее по задолжностям"})
purpose_debt.sort_values(by='Количество записей', ascending=False)

,Количество записей,Среднее по задолжностям
purpose_category,,
операции с недвижимостью,10811,7.23%
операции с автомобилем,4306,9.36%
получение образования,4013,9.22%
проведение свадьбы,2323,8.01%


Можно заметить, что когда люди берут кредит на недвижимость вполне возможно, что ее закроют с сравнению с остальными кредитами. Скорее всего люди не хотят рисковать своим жильем и из-за этого люди закрывают кредит в срок.
